# Pętle

Wykorzystywane wówczas, kiedy kod trzeba wykonać kilka razy dla różnych wartości. W SAS działają tylko w makrze.

Ogólna postać pętli:

```
%do indeks=start %to stop <%by krok>;
kod SAS;
%end;
```

Zmienna `indeks` jest makrozmienną. Wartości `start`, `stop`, `krok` muszą być liczbami całkowitymi.

In [1]:
libname bib "/folders/myfolders/przetwarzanie-danych-SAS/";

SAS Connection established. Subprocess id is 2689



Korzystając z kodu do imputacji dokonamy modyfikacji uzależniając imputowaną wartość od innych zmiennych.

In [2]:
proc means data=bib.bank_braki n nmiss mean median;
var saldo;
output out=saldo_stat median=saldo_mediana;
run;

data _null_;
set saldo_stat;
call symputx("mz_saldo",saldo_mediana);
run;

%put &mz_saldo;

data bib.bank_imputacja;
set bib.bank_braki;
if saldo=. then saldo=&mz_saldo;
run;

proc means data=bib.bank_imputacja n nmiss mean median;
var saldo;
run;


Wykorzystamy w tym celu pętle oraz informacje o stanie cywilnym. W pierwszej kolejności sprawdźmy czy saldo różni się w tak zdefiniowanych grupach.

In [3]:
proc means data=bib.bank_braki n nmiss median;
var saldo;
class stan_cywil;
output out=saldo_stat median=saldo_mediana;
run;

W zbiorze wyjściowym pojawił się także wiersz z wartościami ogółem zatem go usuwamy.

In [4]:
data saldo_stat;
set saldo_stat;
where _type_=1;
run;

Standardowa imputacja wyglądała by następująco i należałoby powtórzyć poniższy kod 3 razy - dla każdej kategorii stanu cywilnego.

In [5]:
data _null_;
set saldo_stat(obs=1);
call symputx("mz_saldo",saldo_mediana);
run;

%put &mz_saldo;

data bib.bank_imputacja;
set bib.bank_braki;
if saldo=. and stan_cywil=1 then saldo=&mz_saldo;
run;

proc means data=bib.bank_imputacja n nmiss median;
var saldo;
class stan_cywil;
output out=saldo_stat median=saldo_mediana;
run;

Możemy sobie ułatwić życie wykorzystując pętlę.

In [10]:
data bib.bank_imputacja;
set bib.bank_braki;
run;

%macro imputacja();

%do i=1 %to 3;

data _null_;
set saldo_stat(obs=&i);
call symputx("mz_saldo",saldo_mediana);
run;

%put &mz_saldo;

data bib.bank_imputacja;
set bib.bank_imputacja;
if saldo=. and stan_cywil=&i then saldo=&mz_saldo;
run;

%end;

%mend;

%imputacja();

In [15]:
proc means data=bib.bank_imputacja n nmiss median;
var saldo;
class stan_cywil;
run;

Możemy jeszcze ulepszyć ten kod dodając makrozmienną z nazwą kategorii oraz automatycznym ustalaniem liczby wierszy w zbiorze z wartościami do zaimputowania.

In [13]:
data bib.bank_imputacja;
set bib.bank_braki;
run;

data _null_;
 set saldo_stat nobs=nobs;
 call symputx("n", nobs);
run;

%macro imputacja();

%do i=1 %to &n;

data _null_;
set saldo_stat(obs=&i);
call symputx("mz_saldo", saldo_mediana);
call symputx("mz_kat", stan_cywil);
run;

%put &mz_saldo;

data bib.bank_imputacja;
set bib.bank_imputacja;
if saldo=. and stan_cywil=&mz_kat then saldo=&mz_saldo;
run;

%end;

%mend;

%imputacja();

## Zadanie

Uzupełnij braki w zmiennej `praca` najczęściej występującą kategorią, a następnie biorąc pod uwagę typ wykonywanej pracy dokonaj imputacji cechy `czas`.